In [12]:
#Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [13]:
#Create spark session
spark = SparkSession.builder \
    .appName("ExploreOFFCSV") \
    .getOrCreate()

In [14]:
csv_path = "../data/raw/openfoodfacts_products.csv"
df = spark.read.csv(
    csv_path,
    header=True,
    sep="\t",      
    inferSchema=True
)

In [15]:
print("=== Nombre de lignes ===")
print(df.count())

=== Nombre de lignes ===
4167065


In [16]:
print("=== Nombre de colonnes ===")
print(len(df.columns))

=== Nombre de colonnes ===
215


In [17]:
print("=== Quelques colonnes ===")
print(df.columns[:50])

=== Quelques colonnes ===
['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'brands_en', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en']


In [18]:
print("=== Schéma inféré ===")
df.printSchema()

=== Schéma inféré ===
root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- brands_en: string (nullable = true)
 |-- categories: string (nullab

In [19]:
# display 20 lines
df.show(20, truncate=True)

+------+--------------------+--------------------+----------+-------------------+---------------+----------------------+----------------+--------------+---------------------+--------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+--------------------+------------+--------------------+--------------------+--------------------+-----

Modèle de données proposé (datamart)

Dimensions (extrait) 
● dim_time(time_sk, date, year, month, day, week, iso_week) 
● dim_brand(brand_sk, brand_name) 
● dim_category(category_sk, category_code, category_name_fr, level, 
parent_category_sk) 
● dim_country(country_sk, country_code, country_name_fr) 
● dim_product 
○ product_sk (PK), code (EAN-13… unique nat. key), product_name, brand_sk, 
primary_category_sk, countries_multi (JSON), effective_from, effective_to, 
is_current 
● (Optionnel) dim_nutri(nutri_sk, nutriscore_grade, nova_group, ecoscore_grade) 

Faits 
● fact_nutrition_snapshot 
○ fact_id (PK), product_sk, time_sk, (mesures pour 100g) energy_kcal_100g, 
fat_100g, saturated_fat_100g, sugars_100g, salt_100g, proteins_100g, 
fiber_100g, sodium_100g, 
○ attributs : nutriscore_grade, nova_group, ecoscore_grade, 
completeness_score (0-1), quality_issues_json (JSON) 
● (Optionnel) bridge_product_category(product_sk, category_sk) si vous conservez le 
N-N. 
Note champs OFF : les mesures existent généralement sous nutriments.<nom>_100g et 
les scores/grades (nutriscore_grade, ecoscore_grade, nova_group) + horodatages 
last_modified_t.

In [11]:
spark.stop()